In [ ]:
import os
import sys
import re
import socket  # 新增：用于网络连通性检查
from pypdf import PdfReader
from google import genai
from google.genai import types
import graphviz
from IPython.display import display, Markdown, Image
from glob import glob

# --- 全局配置 ---
# 1. 文献根目录：请确保此目录存在，且包含您的 PDF 文件
PDF_ROOT_DIR = "./academic_papers" 
os.makedirs(PDF_ROOT_DIR, exist_ok=True)
print(f"文献根目录已设置为: {PDF_ROOT_DIR}")

# 2. 结果输出目录
EXTRACT_OUTPUT_DIR = "./extracted_output"
LLM_OUTPUT_DIR = "./llm_output"
VIS_OUTPUT_DIR = "./vis_output"
os.makedirs(EXTRACT_OUTPUT_DIR, exist_ok=True)
print(f"提取与结果输出目录: {EXTRACT_OUTPUT_DIR}")


# --- LLM 核心指令模板 (省略，请保留您文件中的完整模板) ---
LLM_PROMPT_TEMPLATE = """
【LLM 指令：学术文档分析与定制化翻译】

... (此处保留您之前提供的完整 Prompt 内容) ...

--- 待处理论文文本 ---
"""


# --- 核心环境检查 ---

# 1. 检查 API Key 环境变量
if not os.getenv("GEMINI_API_KEY"):
    display(Markdown("**❌ 致命错误：GEMINI_API_KEY 环境变量未设置！**"))
    display(Markdown("请在终端中执行 `export GEMINI_API_KEY=\"YOUR_KEY\"` 后重启 Notebook Kernel。"))
    sys.exit(1)
else:
    # 仅显示 key 的部分内容，确保它存在且不泄露
    key_snippet = os.getenv("GEMINI_API_KEY")[:4] + "..." + os.getenv("GEMINI_API_KEY")[-4:]
    display(Markdown(f"✅ **API Key 检查成功：** 环境变量已设置 ({key_snippet})。"))


# 2. 检查网络连通性 (尝试连接 Google Public DNS 8.8.8.8)
def check_network_connectivity(host="8.8.8.8", port=53, timeout=5):
    """尝试连接到一个稳定的外部服务器以检查网络是否畅通。"""
    try:
        socket.setdefaulttimeout(timeout)
        # 尝试通过 TCP 连接到 Google DNS 的 53 端口
        socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect((host, port))
        display(Markdown("✅ **网络连通性检查成功：** 能够连接到 Google 服务器。"))
        return True
    except Exception as e:
        display(Markdown(f"❌ **网络连通性检查失败：** 无法连接到 Google 服务器 ({host}:{port})。请检查网络或代理设置。详细错误: {e}"))
        return False

# 运行网络检查
if not check_network_connectivity():
    display(Markdown("--- \n**⚠️ 警告：网络连接存在问题，API 调用可能会失败或超时。**\n---"))
    
# ----------------------------------
# (后续 Cell 2, 3, 4 代码不变)

In [ ]:
# --- Step 1: 选择文件并提取 PDF 内容与资源 ---
import workflow_utils as wf
import os
from glob import glob

# 1. 定义核心目录
PDF_ROOT_DIR = "./academic_papers"          # 输入目录
EXTRACT_OUTPUT_DIR = "./extracted_output"   # 存 Raw Text
LLM_OUTPUT_DIR = "./llm_output"             # 存 LLM Result
VIS_OUTPUT_DIR = "./vis_output"             # 存 HTML + Assets

# 确保目录存在
for d in [EXTRACT_OUTPUT_DIR, LLM_OUTPUT_DIR, VIS_OUTPUT_DIR]:
    os.makedirs(d, exist_ok=True)

# ==========================================
# 2. 文件选择交互逻辑 (补回的部分)
# ==========================================
pdf_files = sorted(glob(os.path.join(PDF_ROOT_DIR, "*.pdf")))

if not pdf_files:
    print(f"❌ 在 {PDF_ROOT_DIR} 目录下未找到 PDF 文件！请检查文件放入位置。")
    SELECTED_PDF_PATH = None
else:
    print("请选择要解读的文献：")
    for i, f in enumerate(pdf_files):
        print(f"[{i+1:2d}] {os.path.basename(f)}")
    print("-" * 40)
    
    # 获取用户输入
    try:
        selection = input("请输入序号并回车 (例如 1): ")
        idx = int(selection) - 1
        if 0 <= idx < len(pdf_files):
            SELECTED_PDF_PATH = pdf_files[idx]
            print(f"\n✅ 已选择: {os.path.basename(SELECTED_PDF_PATH)}")
        else:
            print("❌ 序号无效，请重新运行此单元格。")
            SELECTED_PDF_PATH = None
    except ValueError:
        print("❌ 输入格式错误，请输入数字。")
        SELECTED_PDF_PATH = None

# ==========================================
# 3. 执行提取逻辑
# ==========================================
if 'SELECTED_PDF_PATH' in globals() and SELECTED_PDF_PATH:
    try:
        # 调用 workflow_utils 中的新函数
        # 参数：(PDF路径, 纯文本存储目录, 可视化根目录)
        full_text, raw_txt_path, paper_vis_dir, asset_count = wf.extract_text_and_save_assets_smart(
            SELECTED_PDF_PATH, 
            EXTRACT_OUTPUT_DIR, 
            VIS_OUTPUT_DIR
        )
        
        # 自动生成 LLM 结果的路径
        clean_name = os.path.basename(paper_vis_dir) # 获取论文名文件夹名
        llm_result_path = os.path.join(LLM_OUTPUT_DIR, f"{clean_name}_llm_result.txt")
        
        print(f"\n✅ 提取处理成功！")
        print(f"   📄 纯文本备份: {raw_txt_path}")
        print(f"   🖼️ 图片/表格:  {os.path.join(paper_vis_dir, 'assets')} (共 {asset_count} 张)")
        print(f"   🤖 LLM 预设路径: {llm_result_path}")
        
        # 传递关键变量到后续单元格
        globals()['FULL_TEXT_CONTENT'] = full_text
        globals()['LLM_RESULT_PATH'] = llm_result_path
        globals()['PAPER_VIS_DIR'] = paper_vis_dir # 关键：传递可视化目录路径
        
    except Exception as e:
        print(f"❌ 提取过程中发生错误: {e}")
else:
    print("⚠️ 未选择有效文件，无法继续。")

In [ ]:
# --- Step 2: LLM 翻译 ---
if 'FULL_TEXT_CONTENT' in globals() and FULL_TEXT_CONTENT:
    try:
        # 结果将保存到 llm_output/PaperName_llm_result.txt
        wf.run_smart_analysis(FULL_TEXT_CONTENT, LLM_RESULT_PATH)
        print(f"🎉 翻译完成！结果已保存: {LLM_RESULT_PATH}")
    except Exception as e:
        print(f"❌ 翻译中断: {e}")
else:
    print("🔴 未加载文本内容")

In [ ]:
# --- Step 3: 生成 HTML 报告 ---
from IPython.display import IFrame, display, Markdown

if 'LLM_RESULT_PATH' in globals() and os.path.exists(LLM_RESULT_PATH):
    try:
        # 输入：LLM结果文件(在llm_output)，可视化目录(在vis_output/PaperName)
        # 输出：HTML文件(在vis_output/PaperName/Report.html)
        report_path = wf.generate_html_report(LLM_RESULT_PATH, PAPER_VIS_DIR)
        
        display(Markdown(f"### ✅ 最终报告已生成"))
        display(Markdown(f"**报告路径:** `{report_path}`"))
        display(Markdown(f"**资源路径:** `{os.path.join(PAPER_VIS_DIR, 'assets')}`"))
        
        # 预览
        display(IFrame(src=report_path, width="100%", height="600px"))
        
    except Exception as e:
        print(f"🔴 生成报告失败: {e}")